# 39 장. 보호 속성과 디스크립터 처리하기

## 39.1. 보호 속성 알아보기

In [ ]:
class Klass:
    _name = '가을'
    
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        return self._name

In [ ]:
Klass.__dict__

In [ ]:
Klass._name

In [ ]:
k = Klass('너울')

In [ ]:
k.__dict__

In [ ]:
k.get_name()

In [ ]:
k._name

---

In [ ]:
class Mangling:
    __name = '가을'
    
    def __init__(self, name):
        self.__name = name
        
    def get_name(self):
        return self.__name
    
    @classmethod
    def getAttr(cls):
        return cls.__name

In [ ]:
Mangling.__dict__

In [ ]:
Mangling.__name

In [ ]:
Mangling.getAttr()

In [ ]:
Mangling._Mangling__name

In [ ]:
m = Mangling('맹그링')

In [ ]:
m.__dict__

In [ ]:
m.get_name()

In [ ]:
m.__name

In [ ]:
m._Mangling__name

## 39.2. 프로퍼티(Property)

In [ ]:
print(property)

In [ ]:
isinstance(property, type)

In [ ]:
class Data:
    
    def __init__(self, data):
        self._data = data
        
    @property
    def data(self):
        return self._data

In [ ]:
Data.__dict__

In [ ]:
d = Data(10)

In [ ]:
d.data

In [ ]:
d.__dict__

In [ ]:
d.data = 33

---

In [ ]:
class DataInt:
    
    @property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        if isinstance(value, int):
            self._data = value
        else:
            raise AttributeError('정수값이 아닙니다.')
    
    @data.deleter
    def data(self):
        raise AttributeError('삭제할 수 없습니다.')

In [ ]:
t = DataInt()

In [ ]:
t.__dict__

In [ ]:
t.data = 12

In [ ]:
t.__dict__

In [ ]:
t.data

In [ ]:
try:
    t.data = '123'
except Exception as e:
    print(e)

In [ ]:
try:
    del t.data
except Exception as e:
    print(e)

---

In [ ]:
class my_property:
    
    def __init__(self, fget=None, fset=None, fdel=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        
    def setter(self, fset):
        self.fset = fset
        return self
    
    def deleter(self, fdel):
        self.fdel = fdel
        return self    
    
    def getter(self, fget):
        self.fget = fget
        return self    
    
    def __get__(self, instance, owner):
        return self.fget(instance)
    
    def __set__(self, instance, value):
        if not self.fset:
            raise AttributeError
        return self.fset(instance, value)
    
    def __delete__(self, instance):
        if not self.fdel:
            raise AttributeError
        return self.fdel(instance)

In [ ]:
class OnlyInt:
    
    @my_property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        if isinstance(value, int):
            self._data = value
        else:
            raise AttributeError('정수만 처리합니다.')
            
    @data.deleter
    def data(self):
        raise AttributeError('삭제는 안합니다.')

In [ ]:
c = OnlyInt()

In [ ]:
c.__dict__

In [ ]:
c.data = 999

In [ ]:
c.data

In [ ]:
try:
    c.data = '문자열 갱신'
except Exception as e:
    print(e)

---

In [ ]:
class Double:
    
    def __init__(self, value):
        self.value = value
        
    @property
    def double(self):
        return self.value * 2
    
    @double.setter
    def double(self, value):
        raise AttributeError('갱신할 수 없습니다.')

In [ ]:
d = Double(3)

In [ ]:
d.double

In [ ]:
try:
    d.double = 999
except Exception as e:
    print(e)

## 39.3. 디스크립터(Descriptor) 구조 알아보기

In [ ]:
class Descriptor:
    
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print('get')
        return self.value

In [ ]:
class A:
    value = Descriptor(10)
    value1 = Descriptor(20)

In [ ]:
a = A()

In [ ]:
a.value

In [ ]:
a.value1

---

In [ ]:
class Descriptor_:
    
    def __get__(self, instance, owner):
        print('get')
        if not hasattr(instance, '_value'):
            self.__set__(instance, 0)
        return instance._value
    
    def __set__(self, instance, value):
        print('set')
        instance._value = value

In [ ]:
class B:
    value = Descriptor_()
    value1 = Descriptor_()

In [ ]:
B.__dict__

In [ ]:
b = B()

In [ ]:
b.value

In [ ]:
b.value1

In [ ]:
b.value1 = 999

In [ ]:
b.__dict__

In [ ]:
b.value = 777

In [ ]:
b.__dict__

---

In [ ]:
class Descriptor_c:
    
    def __set_name__(self, owner, name):
        self._name = '_' + name
        
    def __get__(self, instance, owner):
        print('get')
        if not (self._name in instance.__dict__):
            self.__set__(instance, 0)
        return instance.__dict__[self._name]
    
    def __set__(self, instance, value):
        print('set')
        instance.__dict__[self._name] = value

In [ ]:
class C:
    value = Descriptor_c()
    value1 = Descriptor_c()

In [ ]:
c = C()

In [ ]:
c.value

In [ ]:
c.value1

In [ ]:
c.__dict__

In [ ]:
c.value = 100

In [ ]:
c.value1 = 999

In [ ]:
c.__dict__

---

In [ ]:
class Descriptor_old:
    
    def __init__(self, name):
        self._name = '_' + name
        
    def __get__(self, instance, owner):
        print('get')
        if not (self._name in instance.__dict__):
            self.__set__(instance, 0)
        return instance.__dict__[self._name]
    
    def __set__(self, instance, value):
        print('set')
        instance.__dict__[self._name] = value

In [ ]:
class Old:
    value = Descriptor_old('value')
    value1 = Descriptor_old('value1')

In [ ]:
o = Old()

In [ ]:
o.value

In [ ]:
o.value1

In [ ]:
o.value = 999

In [ ]:
o.value1 = 777

In [ ]:
o.value, o.value1

## 39.4. 데이터/비데이터 디스크립터

In [ ]:
def add(x, y):
    return x + y

In [ ]:
add = add.__get__(add)

In [ ]:
add

In [ ]:
add.__func__

In [ ]:
add.__func__(10, 10)

---

In [ ]:
class Descriptor_nd:
    
    def __init__(self, func):
        self._func = func
        
    def __set_name__(self, owner, name):
        self._name = '_' + name
        
    def __get__(self, instance, owner):
        print('get')
        def inner(*args, **kwargs):
            return self._func(*args, **kwargs)
        return inner

In [ ]:
def mul(x, y):
    return x * y

In [ ]:
class Non_data:
    mul = Descriptor_nd(mul)

In [ ]:
nd = Non_data()

In [ ]:
nd.mul

In [ ]:
nd.mul(10, 10)

---

In [ ]:
from functools import partial

In [ ]:
class Descriptor:
    
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        return partial(self.func, instance)

In [ ]:
class A:
    
    @Descriptor
    def sum(self, a, b, c):
        return a + b + c

In [ ]:
A.__dict__

In [ ]:
a = A()

In [ ]:
a.sum(1, 2, 3)

---

In [ ]:
class MutableAttribute:
    
    def __init__(self, value=None):
        self.value = value
        
    def __set_name__(self, owner, name):
        self._name = '_' + name
        
    def __get__(self, instance, owner):
        if self._name in instance.__dict__:
            return instance.__dict__[self._name]
        else:
            return self.value
        
    def __set__(self, instance, value):
        instance.__dict__[self._name] = value
        
    def __delete__(self, instance):
        del self.value

In [ ]:
class ImmutableAttribute:
    
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        return self.func(instance)
        
    def __set__(self, instance, value):
        raise AttributeError('갱신이 불가합니다.')
        
    def __delete__(self, instance):
        raise AttributeError('삭제가 불가합니다.')

In [ ]:
class Circle:
    pi = 3.1415
    radius = MutableAttribute(10)
    diameter = ImmutableAttribute(lambda self: self.radius * 2)
    
    @ImmutableAttribute
    def circumference(self):
        return self.radius * self.pi * 2
    
    @ImmutableAttribute
    def area(self):
        return self.radius**2 * self.pi

In [ ]:
c = Circle()

In [ ]:
c.radius, c.diameter

In [ ]:
c.circumference, c.area

In [ ]:
c.radius = 100

In [ ]:
c.radius

In [ ]:
c.area

In [ ]:
try:
    c.area = 123
except Exception as e:
    print(e)

## 39.5. 디스크립터 속성 자료형 제약하기

In [ ]:
class Field:
    
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise ValueError(f'정수가 아닙니다. 정수값을 넣어주세요. {self.name}')
        instance.__dict__[self.name] = value
        
    def __set_name__(self, owner, name):
        self.name = name        

In [ ]:
class A:
    var = Field()

In [ ]:
a = A()

In [ ]:
a.var = 100

In [ ]:
a.var

In [ ]:
a.__dict__

In [ ]:
a.var = 100.1